In [8]:
from utils import run_shell, run_shell_display

In [20]:
cmd_decode = "kaldi_decoding_scripts//decode_dnn.sh /yhwang/0-Projects/0-kaldi-lstm/2-pytorch-kaldi-cgs/exp/20201021_Pattern_Search/TIMIT_LSTM_fmllr_L2_8bw_16ba_wohcgs_v1/decoding_TIMIT_test_out_dnn2.conf /yhwang/0-Projects/0-kaldi-lstm/2-pytorch-kaldi-cgs/exp/20201021_Pattern_Search/TIMIT_LSTM_fmllr_L2_8bw_16ba_wohcgs_v1/decode_TIMIT_test_out_dnn2 \"/yhwang/0-Projects/0-kaldi-lstm/2-pytorch-kaldi-cgs/exp/20201021_Pattern_Search/TIMIT_LSTM_fmllr_L2_8bw_16ba_wohcgs_v1/exp_files/forward_TIMIT_test_ep*_ck*_out_dnn2_to_decode.ark\""
log_file = "exp/20201021_Pattern_Search/TIMIT_LSTM_fmllr_L2_8bw_16ba_wohcgs_v1/log.log"
run_shell(cmd_decode, log_file)
cmd_res = "./check_res_dec.sh /yhwang/0-Projects/0-kaldi-lstm/2-pytorch-kaldi-cgs/exp/20201021_Pattern_Search/TIMIT_LSTM_fmllr_L2_8bw_16ba_wohcgs_v1/decode_TIMIT_test_out_dnn2"
run_shell(cmd_res,log_file).decode("utf-8")

'%WER 16.7 | 192 7215 | 85.8 10.7 3.5 2.5 16.7 100.0 | -1.371 | /yhwang/0-Projects/0-kaldi-lstm/2-pytorch-kaldi-cgs/exp/20201021_Pattern_Search/TIMIT_LSTM_fmllr_L2_8bw_16ba_wohcgs_v1/decode_TIMIT_test_out_dnn2/score_5/ctm_39phn.filt.sys\n'